In [118]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [119]:
df = pd.read_csv("C:\\Users\\Swayam\\Desktop\\ML\\Excel_Files_dataset\\Data_Train.csv")
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [120]:
df.shape

(10683, 11)

In [121]:
df["Date"] = df['Date_of_Journey'].str.split('/').str[0]
df["Month"] = df['Date_of_Journey'].str.split('/').str[1]
df["Year"] = df['Date_of_Journey'].str.split('/').str[2]

In [122]:
df["Dep_hour"] = df['Dep_Time'].str.split(':').str[0]
df["Dep_minute"] = df['Dep_Time'].str.split(':').str[1]

In [123]:
df['Arrival_Time'] = df['Arrival_Time'].str.split(' ').str[0]

In [124]:
df["Arrival_hour"] = df['Arrival_Time'].str.split(':').str[0]
df['Arrival_Minutes'] = df['Arrival_Time'].str.split(':').str[1]

In [125]:
df["Duration_hour"] = df['Duration'].str.split('h').str[0]

In [126]:
labelencoder=LabelEncoder()

df['Airline']=labelencoder.fit_transform(df['Airline'])
df['Source']=labelencoder.fit_transform(df['Source'])
df['Destination']=labelencoder.fit_transform(df['Destination'])
df['Additional_Info']=labelencoder.fit_transform(df['Additional_Info'])
df['Total_Stops']=labelencoder.fit_transform(df['Total_Stops'])

In [127]:
df.drop(["Date_of_Journey", 'Route', 'Dep_Time', 'Arrival_Time', 'Duration'], axis = 1, inplace= True )

In [128]:
df.head(1)

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_hour,Dep_minute,Arrival_hour,Arrival_Minutes,Duration_hour
0,3,0,5,4,8,3897,24,03,2019,22,20,01,10,2


In [129]:
df[df['Duration_hour']=='5m']

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_hour,Dep_minute,Arrival_hour,Arrival_Minutes,Duration_hour
6474,1,4,3,1,8,17327,6,03,2019,16,50,16,55,5m


In [130]:
df.drop(6474, axis=0, inplace = True)

In [131]:
df['Airline'] = df['Airline'].astype(int)
df['Source'] = df['Source'].astype(int)
df['Destination'] = df['Destination'].astype(int)
df['Additional_Info'] = df['Additional_Info'].astype(int)
df['Date'] = df['Date'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Year'] = df['Year'].astype(int)
df['Dep_hour'] = df['Dep_hour'].astype(int)
df['Dep_minute'] = df['Dep_minute'].astype(int)
df['Arrival_hour'] = df['Arrival_hour'].astype(int)
df['Arrival_Minutes'] = df['Arrival_Minutes'].astype(int)
df['Duration_hour'] = df['Duration_hour'].astype(int)
df['Total_Stops'] = df['Total_Stops'].astype(int)

In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10682 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Airline          10682 non-null  int32
 1   Source           10682 non-null  int32
 2   Destination      10682 non-null  int32
 3   Total_Stops      10682 non-null  int32
 4   Additional_Info  10682 non-null  int32
 5   Price            10682 non-null  int64
 6   Date             10682 non-null  int32
 7   Month            10682 non-null  int32
 8   Year             10682 non-null  int32
 9   Dep_hour         10682 non-null  int32
 10  Dep_minute       10682 non-null  int32
 11  Arrival_hour     10682 non-null  int32
 12  Arrival_Minutes  10682 non-null  int32
 13  Duration_hour    10682 non-null  int32
dtypes: int32(13), int64(1)
memory usage: 709.4 KB


In [133]:
df.isnull().sum()

Airline            0
Source             0
Destination        0
Total_Stops        0
Additional_Info    0
Price              0
Date               0
Month              0
Year               0
Dep_hour           0
Dep_minute         0
Arrival_hour       0
Arrival_Minutes    0
Duration_hour      0
dtype: int64

**Model Applying**

In [134]:
#Dependent and Independent Variables
y = df['Price']
X = df.drop(['Price'], axis = 1)

In [135]:
#Standardizing the independent variable
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [136]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7477, 13), (3205, 13), (7477,), (3205,))

**Linear Regression**

In [137]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [138]:
ycap_train = model.predict(X_train)
ycap_train

array([10756.5093899 ,  5564.11812993, 10471.09499723, ...,
       13736.39958984, 10687.18009511, 13085.54165172])

In [139]:
r2_train = r2_score(y_train, ycap_train)
r2_train

0.42582015530134565

In [140]:
mean_squared_error(y_train, ycap_train)

12246020.007481312

In [141]:
mean_absolute_error(y_train, ycap_train)

2493.385252687528

In [142]:
#Test the generalization on test set
ycap_test = model.predict(X_test)
r2_test = r2_score(y_test, ycap_test)
print(r2_test)

0.4064896397589083


In [143]:
mean_squared_error(y_test, ycap_test)

12520426.758916622

In [144]:
mean_absolute_error(y_test, ycap_test)

2510.4105517198223

**Ridge**

In [145]:
model = Ridge()
parameters = {'alpha':[1,2,3,10,20,30,40,50,60,70,80,90]}
ridgecv = GridSearchCV(model, parameters, cv= 5)
ridgecv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [1, 2, 3, 10, 20, 30, 40, 50, 60, 70, 80,
                                   90]})

In [146]:
y_predtrain = ridgecv.predict(X_train)
y_predtrain

array([10750.92179841,  5589.00549634, 10473.34503643, ...,
       13707.11156603, 10676.37859728, 13065.01345529])

In [147]:
r2_score(y_train, y_predtrain)

0.4257921465982605

In [148]:
mean_squared_error(y_train, y_predtrain)

12246617.372821685

In [149]:
mae = mean_absolute_error(y_train, y_predtrain)
mae

2492.889941997321

In [150]:
y_predtest = ridgecv.predict(X_test)
y_predtest

array([ 9639.46255153,  9887.56086446,  6255.98144431, ...,
        9644.21612654, 10556.86847786, 10619.82650948])

In [151]:
r2_test = r2_score(y_test, y_predtest)
r2_test

0.4065995099596287

In [152]:
mse = mean_squared_error(y_test, y_predtest)
mse

12518108.986737298

In [153]:
mae = mean_absolute_error(y_test, y_predtest)
mae

2509.37833483879

**Lasso**

In [154]:
model = Lasso()

In [155]:
parameters = {"alpha": [1,2,5,10,20,30,40,50,60,70,80,90]}
lassocv = GridSearchCV(model, parameters, cv= 5)
lassocv.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [1, 2, 5, 10, 20, 30, 40, 50, 60, 70, 80,
                                   90]})

In [156]:
y_predtrain = lassocv.predict(X_train)
y_predtrain

array([10751.3195756 ,  5556.63772386, 10476.25452715, ...,
       13722.66369639, 10679.49914685, 13081.73566553])

In [157]:
r2_train = r2_score(y_train,y_predtrain)
mse_train = mean_squared_error(y_train, y_predtrain)
mae_train = mean_absolute_error(y_train, y_predtrain)

In [158]:
y_predtest = lassocv.predict(X_test)

In [159]:
r2_test = r2_score(y_test,y_predtest)
mse_test = mean_squared_error(y_test, y_predtest)
mae_test = mean_absolute_error(y_test, y_predtest)

In [160]:
print("R Square for train dataset: ",r2_train)
print("MSE for train dataset: ",mse_train)
print("MAE Square for train dataset: ",mae_train)
print()
print("R Square for test dataset: ",r2_test)
print("MSE for test dataset: ",mse_test)
print("MAE Square for test dataset: ",mae_test)

R Square for train dataset:  0.4258176910298188
MSE for train dataset:  12246072.565087957
MAE Square for train dataset:  2492.98100077761

R Square for test dataset:  0.40652772071122945
MSE for test dataset:  12519623.42032915
MAE Square for test dataset:  2509.8095051503287


**Elastic Net**

In [161]:
model = ElasticNet()
model.fit(X_train, y_train)

ElasticNet()

In [162]:
y_predtrain = model.predict(X_train)
y_predtrain

array([10498.14646053,  6398.87298387, 10377.53643135, ...,
       12487.84776807, 10219.99253023, 12174.52716707])

In [163]:
r2_train = r2_score(y_train,y_predtrain)
mse_train = mean_squared_error(y_train, y_predtrain)
mae_train = mean_absolute_error(y_train, y_predtrain)

In [164]:
y_predtest = model.predict(X_test)

In [165]:
r2_test = r2_score(y_test,y_predtest)
mse_test = mean_squared_error(y_test, y_predtest)
mae_test = mean_absolute_error(y_test, y_predtest)

In [166]:
print("R Square for train dataset: ",r2_train)
print("MSE for train dataset: ",mse_train)
print("MAE Square for train dataset: ",mae_train)
print()
print("R Square for test dataset: ",r2_test)
print("MSE for test dataset: ",mse_test)
print("MAE Square for test dataset: ",mae_test)

R Square for train dataset:  0.392212202811992
MSE for train dataset:  12962805.283723649
MAE Square for train dataset:  2605.329112321612

R Square for test dataset:  0.37803274681736854
MSE for test dataset:  13120740.532236684
MAE Square for test dataset:  2598.766936501062


**KNN Regressor**

In [167]:
model = KNeighborsRegressor() #Declaring the model
parametrs = {"n_neighbors":[5,7,9,11,13,15,17,19,21,23,25], "metric":["euclidean","cityblock","manhattan",]}
clf = GridSearchCV(model,parametrs,cv=5)
clf.fit(X_train,y_train)
print("The best parameters are: ",clf.best_params_)
print("The best score is: ",clf.best_score_)

The best parameters are:  {'metric': 'cityblock', 'n_neighbors': 5}
The best score is:  0.7823992408547756


In [168]:
y_predtrain = clf.predict(X_train)
y_predtrain

array([ 7739.8,  3846.2, 10677.4, ..., 16067.6, 13344.8, 12466. ])

In [169]:
r2_train = r2_score(y_train,y_predtrain)
mse_train = mean_squared_error(y_train, y_predtrain)
mae_train = mean_absolute_error(y_train, y_predtrain)

In [170]:
y_predtest = clf.predict(X_test)

In [171]:
r2_test = r2_score(y_test,y_predtest)
mse_test = mean_squared_error(y_test, y_predtest)
mae_test = mean_absolute_error(y_test, y_predtest)

In [172]:
print("R Square for train dataset: ",r2_train)
print("MSE for train dataset: ",mse_train)
print("MAE Square for train dataset: ",mae_train)
print()
print("R Square for test dataset: ",r2_test)
print("MSE for test dataset: ",mse_test)
print("MAE Square for test dataset: ",mae_test)

R Square for train dataset:  0.8745748256292457
MSE for train dataset:  2675048.957164638
MAE Square for train dataset:  830.6803263340913

R Square for test dataset:  0.7683701264220033
MSE for test dataset:  4886359.298146646
MAE Square for test dataset:  1094.9697347893916


**SVM**

In [173]:
# Initialize an SVR model
svr = SVR()

# Define the parameter grid for GridSearchCV
param_grid = {
    'kernel': ['linear', 'poly', 'rbf'],
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 1],
    'gamma': ['scale', 'auto']
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5 )

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'C': 100, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}


In [174]:
y_predtrain = clf.predict(X_train)
y_predtrain

array([ 7739.8,  3846.2, 10677.4, ..., 16067.6, 13344.8, 12466. ])

In [175]:
r2_train = r2_score(y_train,y_predtrain)
mse_train = mean_squared_error(y_train, y_predtrain)
mae_train = mean_absolute_error(y_train, y_predtrain)

In [176]:
y_predtest = clf.predict(X_test)
r2_test = r2_score(y_test,y_predtest)
mse_test = mean_squared_error(y_test, y_predtest)
mae_test = mean_absolute_error(y_test, y_predtest)

In [177]:
print("R Square for train dataset: ",r2_train)
print("MSE for train dataset: ",mse_train)
print("MAE Square for train dataset: ",mae_train)
print()
print("R Square for test dataset: ",r2_test)
print("MSE for test dataset: ",mse_test)
print("MAE Square for test dataset: ",mae_test)

R Square for train dataset:  0.8745748256292457
MSE for train dataset:  2675048.957164638
MAE Square for train dataset:  830.6803263340913

R Square for test dataset:  0.7683701264220033
MSE for test dataset:  4886359.298146646
MAE Square for test dataset:  1094.9697347893916


**Gradient Boosting Tech**

In [178]:
'''# Initialize a Gradient Boosting Regressor
gbr = GradientBoostingRegressor()

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [200, 300],
    'learning_rate': [0.1, 0.2],
    'max_depth': [2,3],
    'min_samples_split': [2,3],
    'min_samples_leaf': [1, 2],
    'subsample': [0.9, 1.0]
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=5, verbose=2)

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best Parameters:", grid_search.best_params_)

SyntaxError: incomplete input (4047190779.py, line 1)

In [ ]:
y_predtrain = grid_search.predict(X_train)
y_predtrain

In [ ]:
r2_train = r2_score(y_train,y_predtrain)
mse_train = mean_squared_error(y_train, y_predtrain)
mae_train = mean_absolute_error(y_train, y_predtrain)

In [ ]:
y_predtest = grid_search.predict(X_test)

In [ ]:
r2_test = r2_score(y_test,y_predtest)
mse_test = mean_squared_error(y_test, y_predtest)
mae_test = mean_absolute_error(y_test, y_predtest)

In [ ]:
print("R Square for train dataset: ",r2_train)
print("MSE for train dataset: ",mse_train)
print("MAE Square for train dataset: ",mae_train)
print()
print("R Square for test dataset: ",r2_test)
print("MSE for test dataset: ",mse_test)
print("MAE Square for test dataset: ",mae_test)

**Random Forest**

In [ ]:
'''rfr = RandomForestRegressor()
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [1,2,3,4,5,6],
    'min_samples_split': [2,3,4],
    'min_samples_leaf': [0,1,2,3],
    'bootstrap': [True, False]
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=5, verbose=3)

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)'''

In [ ]:
y_predtrain = grid_search.predict(X_train)
y_predtrain

In [ ]:
r2_train = r2_score(y_train,y_predtrain)
mse_train = mean_squared_error(y_train, y_predtrain)
mae_train = mean_absolute_error(y_train, y_predtrain)

In [ ]:
y_predtest = grid_search.predict(X_test)

In [ ]:
r2_test = r2_score(y_test,y_predtest)
mse_test = mean_squared_error(y_test, y_predtest)
mae_test = mean_absolute_error(y_test, y_predtest)

In [ ]:
print("R Square for train dataset: ",r2_train)
print("MSE for train dataset: ",mse_train)
print("MAE Square for train dataset: ",mae_train)
print()
print("R Square for test dataset: ",r2_test)
print("MSE for test dataset: ",mse_test)
print("MAE Square for test dataset: ",mae_test)